# Real-world Data Wrangling

In this project, you will apply the skills you acquired in the course to gather and wrangle real-world data with two datasets of your choice.

You will retrieve and extract the data, assess the data programmatically and visually, accross elements of data quality and structure, and implement a cleaning strategy for the data. You will then store the updated data into your selected database/data store, combine the data, and answer a research question with the datasets.

Throughout the process, you are expected to:

1. Explain your decisions towards methods used for gathering, assessing, cleaning, storing, and answering the research question
2. Write code comments so your code is more readable

## 1. Gather data

In this section, you will extract data using two different data gathering methods and combine the data. Use at least two different types of data-gathering methods.

### **1.1. Problem Statement**

Air pollution is one of the most critical environmental issues worldwide, impacting human health and quality of life. 
In this project, I aim to explore the relationship between weather conditions (temperature, humidity, wind speed) and air pollution levels (PM2.5, NO₂, O₃) in Rome, Italy. 
The goal is to assess whether certain meteorological factors correlate with variations in pollutant concentrations.

To conduct this analysis, I will gather daily weather data from the OpenWeather API and air quality data from the European Environment Agency (EEA) open data portal. 
These datasets will be merged on date and city for combined analysis.


### **1.2.** Gather at least two datasets using two different data gathering methods

List of data gathering methods:

- Download data manually
- Programmatically downloading files
- Gather data by accessing APIs
- Gather and extract data from HTML files using BeautifulSoup
- Extract data from a SQL database

Each dataset must have at least two variables, and have greater than 500 data samples within each dataset.

For each dataset, briefly describe why you picked the dataset and the gathering method (2-3 full sentences), including the names and significance of the variables in the dataset. Show your work (e.g., if using an API to download the data, please include a snippet of your code). 

Load the dataset programmtically into this notebook.

#### **Dataset 1**

- **Type**: JSON (API response)  
- **Source**: [Open-Meteo Archive API (ERA5)](https://open-meteo.com)  
- **Method**: Programmatically gathered via HTTP request using the `requests` library.  
- **Variables**:  
  * `date` — date of observation (UTC)  
  * `temp_c` — daily average temperature in Celsius  
  * `rhum_pct` — daily average relative humidity (%)  
  * `wind_speed_ms` — daily average wind speed (m/s)  
  * `precip_mm` — daily total precipitation (mm)  


In [30]:
from modules.openmeteo_weather import build_weather_daily

weather_df_daily = build_weather_daily(
    lat=41.9028, 
    lon=12.4964,
    start="2024-01-01",
    end="2024-12-31",
    
)


weather_df_daily.head()

,date,temp_c,rhum_pct,wind_speed_ms,precip_mm
0,2024-01-01,11.841667,85.833333,9.845833,3.5
1,2024-01-02,8.845833,86.458333,7.137500,0.2
2,2024-01-03,13.070833,82.791667,12.275000,0.1
3,2024-01-04,12.237500,82.583333,6.391667,0.0
4,2024-01-05,12.720833,81.500000,14.466667,5.2


#### **Dataset 2**

- **Type**: CSV file generated programmatically from an API query  
- **Source**: [OpenAQ API v3](https://api.openaq.org/)  
- **Method**: Programmatically gathered via HTTP requests using a custom Python module (`openaq_loader.py`). The script retrieves hourly air quality measurements from OpenAQ and aggregates them into daily averages.  
- **Variables**:  
  * `date` — date of observation (UTC)  
  * `pm25` — daily average PM₂.₅ concentration (µg/m³)  
  * `no2` — daily average NO₂ concentration (µg/m³)  


In [31]:
from modules.openaq_loader import OpenAQClient, load_and_aggregate_from_openaq

aq_daily = load_and_aggregate_from_openaq(
    city="Rome",
    parameters=("pm25","no2"),
    start_date="2024-01-01",
    end_date="2024-12-31",
    daily=True,
    api_key="1e4497f55593ef98625dd6d8686aacfbb7bf4661f96b4cb635595927935bf7f7",
    sensors_per_param=2,
    sensor_limit=250,
    verbose=True
)

aq_daily.head(3)

Locations found: 30
Parameter 'pm25': 2 sensors -> [21806, 21846]
Parameter 'no2': 2 sensors -> [21915, 21941]
Fetching days for sensor 21806 (pm25)
Fetching days for sensor 21846 (pm25)
Fetching days for sensor 21915 (no2)
Fetching days for sensor 21941 (no2)


,date,city,pm25,no2
0,2023-12-31,Rome,36.5,39.05
1,2024-01-01,Rome,14.5,39.65
2,2024-01-02,Rome,8.5,31.40


In [32]:
import datetime as dt
import pandas as pd

weather_df_daily['date'] = pd.to_datetime(weather_df_daily['date'], errors='coerce').dt.date
aq_daily['date'] = pd.to_datetime(aq_daily['date'], errors='coerce').dt.date


In [33]:
merged = weather_df_daily.merge(aq_daily, on="date", how="inner")
print(merged.shape)
merged.head()

(365, 8)


,date,temp_c,rhum_pct,wind_speed_ms,precip_mm,city,pm25,no2
0,2024-01-01,11.841667,85.833333,9.845833,3.5,Rome,14.5,39.65
1,2024-01-02,8.845833,86.458333,7.137500,0.2,Rome,8.5,31.40
2,2024-01-03,13.070833,82.791667,12.275000,0.1,Rome,14.5,48.75
3,2024-01-04,12.237500,82.583333,6.391667,0.0,Rome,15.0,39.65
4,2024-01-05,12.720833,81.500000,14.466667,5.2,Rome,9.0,32.80


Optional data storing step: You may save your raw dataset files to the local data store before moving to the next step.

In [34]:
# Save to CSV
merged.to_csv("data/eea_air_quality_rome.csv", index=False)

merged.head()

,date,temp_c,rhum_pct,wind_speed_ms,precip_mm,city,pm25,no2
0,2024-01-01,11.841667,85.833333,9.845833,3.5,Rome,14.5,39.65
1,2024-01-02,8.845833,86.458333,7.137500,0.2,Rome,8.5,31.40
2,2024-01-03,13.070833,82.791667,12.275000,0.1,Rome,14.5,48.75
3,2024-01-04,12.237500,82.583333,6.391667,0.0,Rome,15.0,39.65
4,2024-01-05,12.720833,81.500000,14.466667,5.2,Rome,9.0,32.80


#### **Relationship between Dataset 1 and Dataset 2**

Both datasets refer to environmental conditions in Rome during 2024.  
Dataset 1 contains daily weather variables (temperature, humidity, wind speed, and precipitation) from the Open-Meteo ERA5 reanalysis, while Dataset 2 contains daily air quality measurements (PM₂.₅ and NO₂) from OpenAQ.  
They share a common `date` field, which allows the datasets to be merged for exploratory analysis of the relationship between meteorological conditions and air pollution levels.


## 2. Assess data

Assess the data according to data quality and tidiness metrics using the report below.

List **two** data quality issues and **two** tidiness issues. Assess each data issue visually **and** programmatically, then briefly describe the issue you find.  **Make sure you include justifications for the methods you use for the assessment.**

### Quality Issue 1:
Some PM2.5 values are recorded as `-999`, representing missing or invalid data.

In [35]:
#FILL IN - Inspecting the dataframe visually
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           365 non-null    object 
 1   temp_c         365 non-null    float64
 2   rhum_pct       365 non-null    float64
 3   wind_speed_ms  365 non-null    float64
 4   precip_mm      365 non-null    float64
 5   city           365 non-null    object 
 6   pm25           359 non-null    float64
 7   no2            365 non-null    float64
dtypes: float64(6), object(2)
memory usage: 22.9+ KB


In [36]:
#FILL IN - Inspecting the dataframe programmatically

Issue and justification: *FILL IN*

### Quality Issue 2:
Several entries contain `NaN` values for pollutant concentrations and meteorological data.

In [37]:
#FILL IN - Inspecting the dataframe visually

In [38]:
#FILL IN - Inspecting the dataframe programmatically

Issue and justification: *FILL IN*

### Tidiness Issue 1:
The datasets are stored in separate tables (weather and air quality), requiring a merge on date and city.

In [39]:
#FILL IN - Inspecting the dataframe visually

In [40]:
#FILL IN - Inspecting the dataframe programmatically

Issue and justification: *FILL IN*

### Tidiness Issue 2:
Pollutant types are stored as separate columns, which may need transformation into a long format for analysis.

In [41]:
#FILL IN - Inspecting the dataframe visually

In [42]:
#FILL IN - Inspecting the dataframe programmatically

Issue and justification: *FILL IN*

## 3. Clean data
Clean the data to solve the 4 issues corresponding to data quality and tidiness found in the assessing step. **Make sure you include justifications for your cleaning decisions.**

After the cleaning for each issue, please use **either** the visually or programatical method to validate the cleaning was succesful.

At this stage, you are also expected to remove variables that are unnecessary for your analysis and combine your datasets. Depending on your datasets, you may choose to perform variable combination and elimination before or after the cleaning stage. Your dataset must have **at least** 4 variables after combining the data.

In [43]:
# FILL IN - Make copies of the datasets to ensure the raw dataframes 
# are not impacted

### **Quality Issue 1: FILL IN**

In [44]:
# FILL IN - Apply the cleaning strategy

In [45]:
# FILL IN - Validate the cleaning was successful

Justification: *FILL IN*

### **Quality Issue 2: FILL IN**

In [46]:
#FILL IN - Apply the cleaning strategy

In [47]:
#FILL IN - Validate the cleaning was successful

Justification: *FILL IN*

### **Tidiness Issue 1: FILL IN**

In [48]:
#FILL IN - Apply the cleaning strategy

In [49]:
#FILL IN - Validate the cleaning was successful

Justification: *FILL IN*

### **Tidiness Issue 2: FILL IN**

In [50]:
#FILL IN - Apply the cleaning strategy

In [51]:
#FILL IN - Validate the cleaning was successful

Justification: *FILL IN*

### **Remove unnecessary variables and combine datasets**

Depending on the datasets, you can also peform the combination before the cleaning steps.

In [52]:
#FILL IN - Remove unnecessary variables and combine datasets

## 4. Update your data store
Update your local database/data store with the cleaned data, following best practices for storing your cleaned data:

- Must maintain different instances / versions of data (raw and cleaned data)
- Must name the dataset files informatively
- Ensure both the raw and cleaned data is saved to your database/data store

In [53]:
#FILL IN - saving data

## 5. Answer the research question

### **5.1:** Define and answer the research question 
Going back to the problem statement in step 1, use the cleaned data to answer the question you raised. Produce **at least** two visualizations using the cleaned data and explain how they help you answer the question.

*Research question:* FILL IN from answer to Step 1

In [54]:
#Visual 1 - FILL IN

*Answer to research question:* FILL IN

In [55]:
#Visual 2 - FILL IN

*Answer to research question:* FILL IN

### **5.2:** Reflection
In 2-4 sentences, if you had more time to complete the project, what actions would you take? For example, which data quality and structural issues would you look into further, and what research questions would you further explore?

*Answer:* FILL IN